In [3]:
# 遗传算法
import numpy as np
from deap import base, creator, tools, algorithms
import random

# 定义选择的数量
N = 5000

# 定义适应度函数
def fitness_function(ind):
    # 提取选择的索引
    i_ind = ind[:N]
    j_ind = ind[N:2*N]
    k_ind = ind[2*N:3*N]
    
    # 计算频数
    freq_i = np.bincount(i_ind, minlength=100)
    freq_j = np.bincount(j_ind, minlength=50)
    freq_k = np.bincount(k_ind, minlength=50)

    # 合并频数
    combined_freq = np.concatenate((freq_i, freq_j, freq_k))
    
    # 返回最小频数（作为适应度，取负值进行最大化）
    return np.min(combined_freq),

# 遗传算法参数设置
creator.create("FitnessMin", base.Fitness, weights=(1.0,))  # 最大化
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("indices", random.choices, range(100), k=N)  # i 的选择
toolbox.register("j_indices", random.choices, range(50), k=N)  # j 的选择
toolbox.register("k_indices", random.choices, range(50), k=N)  # k 的选择
toolbox.register("individual", tools.initIterate, creator.Individual, 
                 lambda: toolbox.indices() + toolbox.j_indices() + toolbox.k_indices())
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# 遗传算法执行
population = toolbox.population(n=1000)  # 种群大小
ngen = 200  # 最大迭代次数

for gen in range(ngen):
    # 选择
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # 交叉和变异
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.5:  # 交叉概率
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < 0.3:  # 变异概率
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # 评估新一代个体
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # 替换种群
    population[:] = offspring

# 输出最佳解
fits = [ind.fitness.values[0] for ind in population]
best_idx = np.argmax(fits)
best_ind = population[best_idx]

# 计算最佳解的最小频数
best_freq_i = np.bincount(best_ind[:N], minlength=100)
best_freq_j = np.bincount(best_ind[N:2*N], minlength=50)
best_freq_k = np.bincount(best_ind[2*N:3*N], minlength=50)
combined_best_freq = np.concatenate((best_freq_i, best_freq_j, best_freq_k))
min_freq = np.min(combined_best_freq)

print("最佳最小频数:", min_freq,combined_best_freq)

最佳最小频数: 41 [ 55  45  55  57  44  56  42  44  46  48  48  57  49  53  44  44  43  58
  48  62  52  41  49  44  46  51  54  51  56  56  48  49  51  45  56  46
  54  56  46  51  58  41  44  45  53  54  50  51  42  58  48  59  60  41
  45  55  62  45  53  46  56  52  46  57  62  49  41  53  47  46  52  52
  47  51  51  44  41  45  54  51  48  53  57  47  56  47  46  50  44  48
  46  51  50  47  50  47  53  57  44  52 278 114  96  93 117  93 119 104
  94 114 106  91  99 100  87  89  83  96  87  88  89  88 115  92  95  83
 112  85  88 110  91 108  81 100  85  90  92 100 105 101  88  84  86  97
 104  88 106 104  97  88 540 102  80  84 113  94 100  87  96  94  85  76
  88  72 100 116  95  89  84  83  84 103  97  86  87  79  80  88 102 102
  96  88 103  90  82  75  95 105  86  94  78  82  99  82 102  83  93  91
 103  87]
